# semyondipner

In [5]:
# Data Processing
import numpy as np
import pandas as pd

# Datetime operations
from datetime import datetime as dt
from datetime import timedelta as td

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
PATH = r"./data/"

# Data Downloading
sales_df = pd.read_csv(PATH + "shop_sales" + ".csv")
dates_df = pd.read_csv(PATH + "shop_sales_dates" + ".csv")
price_df = pd.read_csv(PATH + "shop_sales_prices" + ".csv")

In [8]:
sales_df


,item_id,store_id,date_id,cnt
0,STORE_2_085,STORE_2,1,3
1,STORE_2_085,STORE_2,2,8
2,STORE_2_085,STORE_2,3,0
3,STORE_2_085,STORE_2,4,3
4,STORE_2_085,STORE_2,5,0
...,...,...,...,...
81850,STORE_1_727,STORE_1,1815,2
81851,STORE_1_727,STORE_1,1816,3
81852,STORE_1_727,STORE_1,1817,1
81853,STORE_1_727,STORE_1,1818,4


In [11]:
PATH = r"./data/"

# Data Downloading
sales_df = pd.read_csv(PATH + "shop_sales" + ".csv")
dates_df = pd.read_csv(PATH + "shop_sales_dates" + ".csv")
price_df = pd.read_csv(PATH + "shop_sales_prices" + ".csv")

# Data preparation
dates_df["date"] = pd.to_datetime(dates_df["date"])

# item_id
sales_df["item_id"] = sales_df["item_id"].apply(lambda item: int(item[8:]))
price_df["item_id"] = price_df["item_id"].apply(lambda item: int(item[8:]))

# store_id
sales_df["store_id"] = sales_df["store_id"].apply(lambda item: int(item[6:]))
price_df["store_id"] = price_df["store_id"].apply(lambda item: int(item[6:]))

link_df = dates_df[["date_id", "wm_yr_wk", "date"]].copy()
link_df = sales_df.merge(link_df, how="left", on=["date_id"])
link_df = link_df.merge(price_df, how="left", on=["store_id", "item_id", "wm_yr_wk"])
# link_df.drop(columns=["wm_yr_wk", "date_id"], inplace=True)
# df = link_df[["date", "store_id", "item_id", "cnt", "sell_price"]].copy()
df = link_df[["wm_yr_wk", "date_id", "date", "store_id", "item_id", "cnt", "sell_price"]].copy()
del link_df
df["gmv"] = df["cnt"] * df["sell_price"]
df.head()

,wm_yr_wk,date_id,date,store_id,item_id,cnt,sell_price,gmv
0,11101,1,2011-01-29,2,85,3,1.0,3.0
1,11101,2,2011-01-30,2,85,8,1.0,8.0
2,11101,3,2011-01-31,2,85,0,1.0,0.0
3,11101,4,2011-02-01,2,85,3,1.0,3.0
4,11101,5,2011-02-02,2,85,0,1.0,0.0


## Разделение на train_split и valid

In [36]:
PATH = r"./data/"

# Data Downloading
sales_df = pd.read_csv(PATH + "shop_sales" + ".csv")
dates_df = pd.read_csv(PATH + "shop_sales_dates" + ".csv")
price_df = pd.read_csv(PATH + "shop_sales_prices" + ".csv")

# Train
dates_lst_train = df[df["date_id"] < 1819 // 2]["date_id"].unique()
wm_yr_wk_lst_train = df[df["date_id"] < 1819 // 2]["wm_yr_wk"].unique()

# Valid
dates_lst_valid = df[df["date_id"] >= 1819 // 2]["date_id"].unique()
wm_yr_wk_lst_valid = df[df["date_id"] >= 1819 // 2]["wm_yr_wk"].unique()

assert sales_df.shape[0] == (
    sales_df[sales_df.date_id.isin(dates_lst_train)].shape[0]
    + 
    sales_df[sales_df.date_id.isin(dates_lst_valid)].shape[0]
    )

assert dates_df.shape[0] == (
    dates_df[dates_df.date_id.isin(dates_lst_train)].shape[0]
    + 
    dates_df[dates_df.date_id.isin(dates_lst_valid)].shape[0]
    )

assert price_df.shape[0] == (
    price_df[price_df.wm_yr_wk.isin(wm_yr_wk_lst_train)].shape[0]
    + 
    price_df[price_df.wm_yr_wk.isin(wm_yr_wk_lst_valid)].shape[0]
    )

AssertionError: 

In [19]:
print(df.date.min(), df.date.max())
print(f"store_ids: {df.store_id.unique()}")

2011-01-29 00:00:00 2016-01-21 00:00:00
store_ids: [2 3 1]


In [20]:
df.to_excel(r'/Users/semyondipner/Desktop/' + r'data' + r'.xlsx')

# ❓ Вопросы

**KPI:**
1. Сколько у нас всего магазинов?
2. Сколько у нас всего уникальных товаров?
3. Сколько было всего продаж?
4. Сколько денег мы заработали? Какая у нас была выручка?

**Магазины:**
1. В каком магазине происходит больше всего продаж?
2. Как много денег мы зарабатываем с каждого из магазинов?
3. Какой магазин приносит больше всего прибыли и почему?

**Товары:**
1. Какие товары продаются лучше/хуже всего?
2. 

**Магазины & Товары:**
1. Отличается ли топ товаров в зависимости от магазина?
2. 

In [3]:
# Filters
store_ids = [1, 2, 3]
item_ids = df.item_id.unique().tolist()
date1 = dt(year=2015, month=1, day=29)
date2 = dt(year=2016, month=1, day=22)
temp_df = df[
    (df["store_id"].isin(store_ids))
    & (df["item_id"].isin(item_ids))
    & (df["date"] >= date1)
    & (df["date"] <  date2)
].copy()
temp_df['date'] = temp_df['date'].dt.strftime('%Y-%m-%d')

In [4]:
temp_df.agg(
    num_uniq_shops=('store_id', 'nunique'),
    num_uniq_items=('item_id', 'nunique'),
    total_sales=('cnt', 'sum'),
    total_gmv=('gmv', 'sum')
)

,store_id,item_id,cnt,gmv
num_uniq_shops,3.0,NaN,NaN,NaN
num_uniq_items,NaN,31.0,NaN,NaN
total_sales,NaN,NaN,338111.0,NaN
total_gmv,NaN,NaN,NaN,571247.52


## KPI

In [5]:
kpi_metrics = ( 
    temp_df
    .agg({
        "store_id": "nunique",
        "item_id": "nunique",
        "cnt": "sum",
        "gmv": "sum"
    })
    .rename({
        "store_id": "num_uniq_shops",
        "item_id": "num_uniq_items",
        "cnt": "total_sales",
        "gmv": "total_gmv"
    })
    .to_dict()
)
kpi_metrics

{'num_uniq_shops': 3.0,
 'num_uniq_items': 31.0,
 'total_sales': 338111.0,
 'total_gmv': 571247.52}

## Tables

In [6]:
table_shop_sales = (
    temp_df
    .groupby(["store_id"])
    .agg(
        uniq_item_sale=('item_id', 'nunique'),
        sales=('cnt', 'sum'),
        gmv=('gmv', 'sum')
    )
    .sort_values("sales", ascending=False)
    .reset_index()
    .to_dict('split')
)

table_item_sales = (
    temp_df
    .groupby(["item_id"])
    .agg(
        uniq_item_sale=('store_id', 'nunique'),
        sales=('cnt', 'sum'),
        gmv=('gmv', 'sum')
    )
    .sort_values("sales", ascending=False)
    .reset_index()
    .to_dict('split')
)

table_shop_item_sales = (
    temp_df
    .groupby(["store_id", "item_id"])
    .agg(
        sales=('cnt', 'sum'),
        gmv=('gmv', 'sum')
    )
    .sort_values("sales", ascending=False)
    .reset_index()
    .to_dict('split')
)

## Linecharts

In [7]:
from typing import List

def prepare_df(df: pd.DataFrame, index_list: List[str]):
    """
    Prepare DF for Anomaly Detection.

    —— Example 1 —— ( index_list = ["time"] )
    Input:
                  time  DAU  ...  Т-Прибыль
        0   2023-04-01  388  ...     186311
        1   2023-04-02  356  ...     200867
        2   2023-04-03  332  ...     126016
        3   2023-04-04  362  ...     165649
        4   2023-04-05  364  ...     151476
        ...

    Output:
                metric        time   value
        0          DAU  2023-04-01     388
        1          DAU  2023-04-02     356
        2          DAU  2023-04-03     332
        3          DAU  2023-04-04     362
        4          DAU  2023-04-05     364
        ..         ...         ...     ...
        235  Т-Прибыль  2023-04-26  408807
        236  Т-Прибыль  2023-04-27  366919
        237  Т-Прибыль  2023-04-28  484113
        238  Т-Прибыль  2023-04-29  470874
        239  Т-Прибыль  2023-04-30  252187

    —— Example 2 —— ( index_list = ["service", "time"] )
    Input:
                    time               service  DAU  ...   Т-Прибыль
        0     2023-04-01                 1xbet   75  ...       82616
        1     2023-04-01         1xbet_invoice    3  ...        2448
        2     2023-04-01     1xbetsub2_wp_sale   27  ...           0
        3     2023-04-01                 activ    5  ...          39
        4     2023-04-01  addressless_transfer    1  ...           0

    Output:
                  metric     service        time  value
        0            DAU       1xbet  2023-04-01     75
        1            DAU       1xbet  2023-04-02     63
        2            DAU       1xbet  2023-04-03     50
        3            DAU       1xbet  2023-04-04     68
        4            DAU       1xbet  2023-04-05     71
        ...          ...         ...         ...    ...
        13067  Т-Прибыль  wpsub_sale  2023-04-30 -49349
        13068  Т-Прибыль      xsolla  2023-04-23      0
        13069  Т-Прибыль      zepter  2023-04-04    -11
        13070  Т-Прибыль      zepter  2023-04-05    -31
        13071  Т-Прибыль      zepter  2023-04-14    -85
    """
    level_counter = len(index_list)
    df = df.set_index(index_list).stack().reset_index().rename(columns={
        f"level_{level_counter}": "metric",
        0: "value"
    })
    df = df.groupby(["metric", *index_list])["value"].sum().reset_index()
    return df.copy()

def get_anomalies(df: pd.DataFrame, groupby_list: List[str]):
    """
    Input:
        Prepared DF.
    Output:
        pd.DataFrame with anomalies.
    """
    # ROLLING MEAN
    df["rolling_mean"] = (
        df
        .groupby(groupby_list)
        ['value']
        .rolling(5)
        .mean()
        .reset_index(drop=True)
    )

    # STD
    df = df.merge(
        right=round(df.groupby(groupby_list)["value"].std()).reset_index()
        .rename(columns={"value": "std"}), on=groupby_list, how="left")

    df = df[df["std"] > 1].copy()
    # Upper/Lower Bounds
    df["upper_bound"] = df["rolling_mean"] + (df["std"] * 2.57)
    df["lower_bound"] = df["rolling_mean"] - (df["std"] * 2.57)

    df["indicator"] = np.where(
        df["value"] > df["upper_bound"], 1,
        np.where(
            df["value"] < df["lower_bound"], -1,
            0
        )
    )
    df["deviation%"] = round(((df["value"] / df["rolling_mean"]) - 1) * 100)
    # df[df["indicator"] != 0].copy()
    return df

In [13]:
dynamics_df = temp_df.groupby(["date"]).agg({col: "sum" for col in ["cnt", "gmv"]}).reset_index()
dynamics_df = prepare_df(dynamics_df, index_list=["date"])
anomalies_df = get_anomalies(dynamics_df, groupby_list=["metric"])
anomalies_df.drop(columns=["std"], inplace=True)
sales_dynamics = (
    anomalies_df
    [anomalies_df["metric"] == "cnt"]
    .reset_index(drop=True)
    .drop(columns=["metric"])
    .to_dict('split')
)
gmv_dynamics = (
    anomalies_df
    [anomalies_df["metric"] == "gmv"]
    .reset_index(drop=True)
    .drop(columns=["metric"])
    .to_dict('split')
)

In [9]:
table_shop_sales

{'index': [0, 1, 2],
 'columns': ['store_id', 'uniq_item_sale', 'sales', 'gmv'],
 'data': [[2, 15, 136870, 216235.15],
  [3, 15, 126750, 230840.41999999998],
  [1, 15, 74491, 124171.95]]}

In [10]:
analytics_dashboard_json = {
    "kpi": kpi_metrics,
    "tables": {
        "table_shop_sales": table_shop_sales,
        "table_item_sales": table_item_sales,
        "table_shop_item_sales": table_shop_item_sales
    },
    "dynamics": {
        "sales_dynamics": sales_dynamics,
        "gmv_dynamics": gmv_dynamics
    }
}

In [14]:
gmv_dynamics

,date,value,rolling_mean,upper_bound,lower_bound,indicator,deviation%
0,2015-01-29,1154.47,NaN,NaN,NaN,0,NaN
1,2015-01-30,1612.25,NaN,NaN,NaN,0,NaN
2,2015-01-31,2078.53,NaN,NaN,NaN,0,NaN
3,2015-02-01,1968.21,NaN,NaN,NaN,0,NaN
4,2015-02-02,1540.81,1670.854,2678.294,663.414,0,-8.0
...,...,...,...,...,...,...,...
353,2016-01-17,1830.65,1548.580,2556.020,541.140,0,18.0
354,2016-01-18,1287.87,1554.998,2562.438,547.558,0,-17.0
355,2016-01-19,1231.53,1563.830,2571.270,556.390,0,-21.0
356,2016-01-20,1219.27,1503.748,2511.188,496.308,0,-19.0


In [11]:
import json

with open("test_nested.json", "w") as outfile:
    json.dump(analytics_dashboard_json, outfile)